In [ ]:
!pip install sentence-transformers
!pip install transformers

In [2]:
from google.colab import drive
import os

drive.mount('/content/drive')
os.chdir("drive/MyDrive/project/soma")

Mounted at /content/drive


In [3]:
import pandas as pd
import torch
import torch.nn.functional as F

from transformers import pipeline
from sentence_transformers import SentenceTransformer

In [4]:
reviews = pd.read_csv("tokyo_hotels_review.csv")
keywords = ["exploring","relaxing","come along","partaking"]

reviews.head()

,Unnamed: 0,cons_translated,languagecode,user_new_badges,helpful_vote_count,is_incentivised,reviewng,is_trivial,hotel_id,average_score,...,pros,is_moderated,title,anonymous,title_translated,review_hash,pros_translated,author,tags,travel_purpose
0,0,NaN,en-gb,[],0,0,1,0,245830,3.2,...,"The staff were very friendly, and helped me pr...",0,Good!,NaN,NaN,fdac70a776dd4999,NaN,"{'city': '', 'countrycode': 'jp', 'type': 'sol...",[],business
1,1,NaN,en-us,[],0,0,1,0,245830,3.6,...,This is a fantastic hotel. Although the rooms ...,0,A great Tokyo gem 💎 q,NaN,NaN,0a000040c2fc9ed4,NaN,"{'countrycode': 'nl', 'city': '', 'helpful_vot...",[],business
2,2,NaN,en-gb,[],0,0,1,0,245830,3.2,...,Location is incredible in terms of public tran...,0,Very good value for money,NaN,NaN,0818b510bb4c8d01,NaN,"{'nr_reviews': 0, 'name': 'Aego', 'avatar': 'h...",[],business
3,3,NaN,en-gb,[],1,0,1,0,245830,2.8,...,"excellent location, very nice rooms",0,"nice hotel, very close to the heart of Tokyo",NaN,NaN,f8e58c7dd0b41eaf,NaN,"{'type_string': 'Solo traveller', 'type': 'sol...",[],leisure
4,4,NaN,en-us,[],0,0,1,0,245830,3.2,...,"near from the station and a clean fascility, a...",0,Very good,NaN,NaN,2efbb2b95e2f4ea3,NaN,"{'nr_reviews': 0, 'name': 'Anna', 'age_group':...",[],leisure


In [5]:
reviews[["title", "pros", "cons"]]

,title,pros,cons
0,Good!,"The staff were very friendly, and helped me pr...","The room was quite small, and the bed wasn't p..."
1,A great Tokyo gem 💎 q,This is a fantastic hotel. Although the rooms ...,"Rooms are a bit too small, but still good valu..."
2,Very good value for money,Location is incredible in terms of public tran...,-
3,"nice hotel, very close to the heart of Tokyo","excellent location, very nice rooms",NaN
4,Very good,"near from the station and a clean fascility, a...",it could be much better if there is a small si...
...,...,...,...
367,Exceptional,NaN,NaN
368,Exceptional,NaN,NaN
369,Very good,NaN,NaN
370,Pleasant,NaN,NaN


In [6]:
data = reviews[["hotel_id","title", "pros", "cons"]].fillna("").values.tolist()
hotel = list(d[0] for d in data)
text = list(' '.join(d[1:]).replace('\n', '') for d in data)

text[:10]

["Good! The staff were very friendly, and helped me print important documentation for my onward travel. Room itself was also well equipped. The room was quite small, and the bed wasn't particularly comfy.",
 'A great Tokyo gem 💎 q This is a fantastic hotel. Although the rooms are tiny, they have all the amenities you need at the lobby. \n\nThe staff keeps the rooms impeccable, and they are very polite and welcoming. \n\nA huge plus: the hotel is 3 mins away from Shinjuku station so you can get to your clients very easily. Rooms are a bit too small, but still good value for money. I would have loved to have a good breakfast option at the lobby.',
 'Very good value for money Location is incredible in terms of public transportation, restaurants, entertainment \n.. everything you may want is either there or can be easily reached .\n\nPrice quality is perfect with all services needed, but so not expect a big fancy room (which is totally understandable given the price)... -',
 'nice hotel, v

# Method 1: BART 모델을 활용

https://huggingface.co/facebook/bart-large-mnli

In [12]:
from transformers import pipeline
from tqdm import tqdm

In [9]:
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli", device=0)

Downloading:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [10]:
sequence_to_classify = text[0]
candidate_labels = keywords
classifier(sequence_to_classify, candidate_labels)

{'labels': ['partaking', 'come along', 'relaxing', 'exploring'],
 'scores': [0.38958147168159485,
  0.27416127920150757,
  0.25158506631851196,
  0.08467221260070801],
 'sequence': "Good! The staff were very friendly, and helped me print important documentation for my onward travel. Room itself was also well equipped. The room was quite small, and the bed wasn't particularly comfy."}

In [14]:
sequence_to_classify = text
candidate_labels = keywords
res = classifier(sequence_to_classify, candidate_labels)

res[0]

{'labels': ['partaking', 'come along', 'relaxing', 'exploring'],
 'scores': [0.38958147168159485,
  0.27416127920150757,
  0.25158506631851196,
  0.08467221260070801],
 'sequence': "Good! The staff were very friendly, and helped me print important documentation for my onward travel. Room itself was also well equipped. The room was quite small, and the bed wasn't particularly comfy."}

In [19]:
high_prob = list(filter(lambda x: max(x["scores"]) > 0.6, res))
high_prob[:5]

[{'labels': ['relaxing', 'come along', 'partaking', 'exploring'],
  'scores': [0.7167568206787109,
   0.15600106120109558,
   0.08875059336423874,
   0.038491543382406235],
  'sequence': 'Fabulous Great locations, very quite, right next to Shinjuku Train Station. No waiting for elevators. Just all good Room small like always in Japan... Otherwise nothing negative to say at least not for the low rate due to Covid.'},
 {'labels': ['relaxing', 'come along', 'exploring', 'partaking'],
  'scores': [0.7652678489685059,
   0.10704566538333893,
   0.08139637112617493,
   0.046290136873722076],
  'sequence': 'Fabulous All staff were really friendly and patient, even with my bad Japanese! Location close to Shinjuku station is great if you have lots of luggage or tired from a long trip. The room was comfortable, and being able to watch Netflix on the TV was really relaxing during the evening. Also, the water system is purified so you can drink from it but also feels great on your skin. My skin fe

# Method 2: Sentence Transformer를 활용

https://www.sbert.net/index.html

In [22]:
embedder = SentenceTransformer("all-mpnet-base-v2", device="cuda")

In [23]:
emb_vecs = embedder.encode(text, convert_to_tensor=True)
emb_vecs.shape

torch.Size([372, 768])

In [25]:
keyword_vecs = embedder.encode(keywords, convert_to_tensor=True)
keyword_vecs

tensor([[-0.0020, -0.0284, -0.0465,  ...,  0.0300,  0.0401,  0.0102],
        [-0.0692, -0.0669, -0.0363,  ...,  0.0032, -0.0006, -0.0552],
        [-0.0091,  0.0586, -0.0073,  ...,  0.0155, -0.0275, -0.0220],
        [ 0.0348, -0.0155, -0.0271,  ..., -0.0158,  0.0960, -0.0317]],
       device='cuda:0')

In [32]:
cos_sims = torch.cat([F.cosine_similarity(emb_vecs, keyword_vecs[i]) for i in range(4)]).view(-1, 4)
cos_sims

tensor([[0.1841, 0.1552, 0.1701, 0.1498],
        [0.1620, 0.1569, 0.1815, 0.0841],
        [0.1252, 0.1520, 0.1404, 0.1299],
        ...,
        [0.0234, 0.1197, 0.0873, 0.0310],
        [0.1241, 0.1382, 0.1359, 0.1359],
        [0.1359, 0.1667, 0.2063, 0.1359]], device='cuda:0')

In [35]:
cos_sims.argmax(dim=1)

tensor([0, 2, 1, 3, 3, 0, 3, 2, 3, 3, 0, 2, 1, 3, 3, 0, 0, 3, 3, 1, 1, 0, 2, 3,
        0, 3, 2, 1, 2, 0, 0, 1, 2, 1, 1, 1, 0, 3, 0, 3, 3, 2, 2, 2, 2, 1, 1, 2,
        1, 2, 2, 1, 2, 0, 3, 3, 0, 3, 2, 1, 1, 0, 3, 0, 3, 2, 1, 0, 2, 0, 1, 1,
        2, 3, 3, 0, 3, 2, 0, 1, 2, 2, 2, 0, 1, 0, 2, 2, 2, 3, 0, 0, 0, 0, 0, 3,
        1, 3, 0, 2, 1, 3, 2, 0, 0, 1, 0, 1, 0, 3, 3, 2, 1, 1, 1, 2, 3, 1, 0, 1,
        1, 2, 0, 3, 0, 3, 1, 2, 1, 1, 2, 2, 1, 2, 2, 0, 0, 0, 1, 0, 1, 3, 2, 1,
        1, 3, 0, 3, 3, 3, 3, 0, 1, 1, 0, 3, 3, 3, 2, 3, 1, 1, 3, 1, 1, 2, 3, 1,
        3, 2, 1, 1, 1, 3, 1, 3, 0, 2, 0, 2, 2, 2, 3, 0, 0, 2, 1, 1, 1, 3, 3, 2,
        3, 2, 3, 2, 3, 0, 1, 0, 0, 0, 1, 1, 3, 3, 1, 2, 0, 3, 1, 3, 3, 2, 2, 0,
        3, 1, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 1, 0, 1, 0, 2, 3, 0, 0, 1,
        0, 3, 1, 2, 2, 0, 1, 2, 3, 1, 3, 2, 0, 0, 2, 3, 1, 1, 3, 1, 3, 2, 2, 3,
        1, 1, 3, 3, 0, 3, 3, 0, 2, 3, 0, 1, 1, 0, 2, 1, 2, 0, 3, 0, 2, 3, 0, 3,
        2, 3, 0, 2, 0, 3, 1, 2, 3, 3, 3,

In [39]:
sbert_res = [keywords[i.item()] for i in cos_sims.argmax(dim=1)]
sbert_res[:10]

['exploring',
 'come along',
 'relaxing',
 'partaking',
 'partaking',
 'exploring',
 'partaking',
 'come along',
 'partaking',
 'partaking']

In [40]:
bart_res = list(map(lambda x: x["labels"][0], res))
bart_res[:10]

['partaking',
 'come along',
 'come along',
 'come along',
 'come along',
 'partaking',
 'partaking',
 'relaxing',
 'partaking',
 'come along']

In [41]:
sum(i==j for i, j in zip(sbert_res, bart_res)) / len(sbert_res)

0.2849462365591398